In [1]:
import pandas as pd

### Paso 1: Inspeccionamos el Dataset, para estudiar la mejor forma de cargarlo

In [3]:
with open('../Data/dataset_SCL.csv') as file:
    n_rows = len(file.readlines())
print (f'Numero de Filas: {n_rows}')

Numero de Filas: 68207


* Con 68 mil filas el dataset facilmente cabe directamente en memoria y no requiere de procesamiento adicional, estandarizando las columnas con valores mixtos

In [8]:
data_vuelos = pd.read_csv('../Data/dataset_SCL.csv', dtype= {'Vlo-I':str, 'Vlo-O':str})
matriz_vuelos = data_vuelos.copy()

In [9]:
matriz_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


* Existe un campo vacio en Vlo-O, se revisa

In [22]:
matriz_vuelos[matriz_vuelos['Vlo-O'].isna()==True]

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,NaN,SCEL,SPJC,56R,19,1,2017,Jueves,I,Latin American Wings,Santiago,Lima
